In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 51 kB/s 
     |████████████████████████████████| 199 kB 51.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=43cbd7640bca567c3d7e26f76a5ae281abaf9446262b64ce915a4047ac3333cd
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz

--2022-10-19 01:10:50--  https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274079476 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.0-bin-hadoop2.tgz’

spark-3.3.0-bin-had 100%[===================>] 261.38M   170MB/s    in 1.5s    

2022-10-19 01:10:59 (170 MB/s) - ‘spark-3.3.0-bin-hadoop2.tgz’ saved [274079476/274079476]



In [4]:
!tar xf spark-3.3.0-bin-hadoop2.tgz

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop2"

In [6]:
!pip install petastorm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 285 kB 11.5 MB/s 
     |████████████████████████████████| 44 kB 2.0 MB/s 


In [7]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import findspark
findspark.init()

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .config("spark.csci316.pandas.SVM", "false") \
    .getOrCreate()
spark

In [11]:
#Read the csv file
data = spark.read.csv("/content/drive/MyDrive/cleanWithHeader.csv", inferSchema=True, header=True)

In [12]:
from petastorm.spark import SparkDatasetConverter, make_spark_converter
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import math

from IPython import embed
from matplotlib import colors
from matplotlib import pyplot as plt
from sklearn import datasets
import numpy as np
from scipy.special import logsumexp
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

# tensorflow hub
import tensorflow_hub as hub
# tensor flow module
import tensorflow as tf
import tensorflow_probability as tfp

# matplotlib
from matplotlib import colors
from matplotlib import pyplot as plt

# word vectorizor
# first converts the text into a matrix of word counts
# then transforms these counts by normalizing them based on the term frequency
from sklearn.feature_extraction.text import TfidfVectorizer

# used to create word encoders
from sklearn import preprocessing

/usr/local/lib/python3.7/dist-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem


In [13]:
data.show(10)

+-------+---------+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+-----------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-------------------------+---------------+----------+-------------------------+
|     id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|default_ind|revol_util_impute

In [14]:
#Re-arrange the dataset
data_reordered = data.select("id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","int_rate","installment","annual_inc",
                             "dti","delinq_2yrs","inq_last_6mths","open_acc","pub_rec","revol_bal","total_acc","out_prncp","out_prncp_inv",
                             "total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries",
                             "collection_recovery_fee","revol_util_imputed","last_pymnt_amnt_imputed","collections_12_mths_ex_med_imputed",
                             "policy_code_imputed","tot_coll_amt_imputed","tot_cur_bal_imputed","total_rev_hi_lim_imputed","pymnt_plan_index",
                             "application_type_index","acc_now_delinq_index","grade_index","purpose_index","home_ownership_index",
                             "emp_length_index","verification_status_index","sub_grade_index","term_index","initial_list_status_index", "default_ind")

In [15]:
df_train, df_val = data_reordered.randomSplit([0.7, 0.3], seed=135)

df_train = df_train.repartition(2)
df_val = df_val.repartition(2)

In [16]:
df_train.show()

+--------+---------+---------+-----------+---------------+--------+-----------+----------+-----+-----------+--------------+--------+-------+---------+---------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------------+-----------------------+----------------------------------+-------------------+--------------------+-------------------+------------------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-------------------------+---------------+----------+-------------------------+-----------+
|      id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|annual_inc|  dti|delinq_2yrs|inq_last_6mths|open_acc|pub_rec|revol_bal|total_acc|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|revol_util_imputed|last_pym

In [17]:
print(f"train: {df_train.count()}, val: {df_val.count()}")

train: 598745, val: 256998


In [18]:
train_dataset = df_train.toPandas()

In [19]:
test_dataset = df_val.toPandas()

In [20]:
x_train = train_dataset.iloc[:,0:42]
y_train = train_dataset.iloc[:,-1]

In [21]:
x_test = test_dataset.iloc[:,0:42]
y_test = test_dataset.iloc[:,-1]

In [22]:
model = tf.estimator.LinearClassifier(
    n_classes = 2,
    model_dir="ongoing/train", 
    feature_columns=x_train)

In [29]:
FEATURES = ["id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","int_rate","installment","annual_inc",
                             "dti","delinq_2yrs","inq_last_6mths","open_acc","pub_rec","revol_bal","total_acc","out_prncp","out_prncp_inv",
                             "total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries",
                             "collection_recovery_fee","revol_util_imputed","last_pymnt_amnt_imputed","collections_12_mths_ex_med_imputed",
                             "policy_code_imputed","tot_coll_amt_imputed","tot_cur_bal_imputed","total_rev_hi_lim_imputed","pymnt_plan_index",
                             "application_type_index","acc_now_delinq_index","grade_index","purpose_index","home_ownership_index",
                             "emp_length_index","verification_status_index","sub_grade_index","term_index","initial_list_status_index"]
LABEL = "default_ind"
def get_input_fn(x_train,y_train, num_epochs=None, n_batch = 128, shuffle=True):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
       x=pd.DataFrame({k: x_train[k].values for k in FEATURES}),
       y = pd.Series(y_train.values),
       batch_size=n_batch,   
       num_epochs=num_epochs,
       shuffle=shuffle)

In [30]:
model.train(input_fn=get_input_fn(x_train, y_train, 
                                      num_epochs=None,
                                      n_batch = 128,
                                      shuffle=False),
                                      steps=1000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


ValueError: ignored